# Generating ground truth for the problem

For each question - anwser / terms of condition point we shall generate 5 relevant questions. We will use LLMs to generate the questions.

In [24]:
import requests 
import json

with open('../data/documents_olx_2024_08_25.json', 'r',  encoding='utf8') as f_in:
     documents = json.load(f_in)

In [25]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [26]:
documents = [item['documents'] for item in documents]
documents = [item for sublist in documents for item in sublist]

In [27]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [29]:
len(documents)

90

In [30]:
from collections import defaultdict

In [31]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [32]:
len(hashes), len(documents)

(90, 90)

In [33]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [34]:
import json

In [36]:
with open('../data/documents-olx-2024_08_25-with-ids.json', 'wt', encoding='utf8') as f_out:
    json.dump(documents, f_out, indent=2, ensure_ascii=False)
    

In [37]:
!head ../data/documents-olx-2024_08_25-with-ids.json

[
  {
    "text": "Krok 1. Załóż konto na OLX poprzez:\nstronę internetową OLX — kliknij Twoje konto → Załóż konto lub kliknij tutaj.\naplikację mobilną OLX — kliknij Konto → Zaloguj się → Załóż konto.\nJeśli grafika jest nieczytelna, kliknij prawy przycisk myszy i wybierz „Otwórz grafikę w nowej karcie\".\nKrok 2. Wpisz swój adres e-mail.\nKrok 3. Wpisz hasło, które ma minimum 8 znaków, w tym przynajmniej jedną dużą i małą literę, jedną cyfrę oraz znak specjalny.\nKrok 4. Kliknij Zarejestruj się.\nKrok 5. Przejdź do skrzynki mailowej i w wiadomości od OLX kliknij przycisk Aktywuj konto.\nJeśli nie masz linku aktywacyjnego — napisz do nas.\nNa jednym koncie OLX nie można sprzedawać przedmiotów prywatnych i przedmiotów w ramach prowadzenia firmy. Do każdego typu sprzedaży (prywatnie, jako firma) wymagane jest oddzielne konto.\nKonto na OLX możesz założyć za pomocą konta na Facebooku. Wystarczy, że klikniesz tutaj. Następnie wybierz przycisk Kontynuuj z Facebook i postępuj zgodnie z wyśw

In [72]:
prompt_template = """
Jesteś pracownikiem działu obsługi klienta w firmie OLX będącej platformą ogłoszeniową.
Otrzymałeś zadanie aby do kazdego ZAGADNIENIE z FAQ lub regulaminu stworzyć 5 pytań, które 
mógłby zadać klient platformy OLX.
Każde ZAGADNIENIE powinno zawierać odpowiedź na pytanie. Pytania powinny być kompletne i nie za krótkie. Jeśli to możliwe, użyj jak najmniej słów z oryginalnego ZAGADNIENIE i ODPOWIEDŹ.

Pytanie:

SEKCJA: {section}
ZAGADNIENIE: {question}
ODPOWIEDŹ: {text}

Odpowiedź powinna być w parsowalnym formacie JSON. Nie używaj bloków kodu:

["pytanie1", "pytanie2", ..., "pytanie"]
""".strip()

In [73]:
from openai import OpenAI
client = OpenAI()

In [74]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [75]:
questions = generate_questions(documents[0])

In [76]:
questions


'["Jak mogę założyć konto na stronie internetowej OLX?", "Jak założyć konto na OLX za pomocą aplikacji mobilnej?", "Jakie wymagania musi spełniać hasło przy tworzeniu konta na OLX?", "Co zrobić, jeśli nie mogę znaleźć linku aktywacyjnego po rejestracji na OLX?", "Czy mogę używać jednego konta OLX do sprzedaży prywatnej i firmowej?"]'

In [77]:
from tqdm.auto import tqdm

In [78]:
results = {}

In [79]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

100%|██████████| 90/90 [03:07<00:00,  2.08s/it]


In [82]:
results

In [58]:
import pickle

In [81]:
with open('../data/results.bin', 'wb') as f_out:
    pickle.dump(results, f_out)

In [83]:
with open('../data/results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [84]:
results['653d264c']

'["Na jakiej podstawie OLX podejmuje decyzje o blokadzie lub zawieszeniu treści lub Konta?",\n "Jakie polityki i procedury moderacyjne są stosowane przez Grupę OLX?",\n "W jaki sposób OLX identyfikuje treści lub działania naruszające Regulamin?",\n "Jakie są główne cele opracowanych przez OLX polityk i procedur moderacyjnych?",\n "Dlaczego ważne jest przestrzeganie regulaminu i przepisów prawa przez użytkowników OLX?"]'

In [88]:
import re 
parsed_resulst = {}

for doc_id, json_questions in results.items():
    json_questions = re.sub(r'```json', '', json_questions)
    json_questions = re.sub(r'```', '', json_questions)
    parsed_resulst[doc_id] = json.loads(json_questions)

In [89]:
parsed_resulst

{'2ddb5f16': ['Jak mogę założyć konto na OLX przez stronę internetową?',
  'Jakie są kroki potrzebne do założenia konta w aplikacji mobilnej OLX?',
  'Jakie wymagania musi spełniać hasło podczas zakładania konta na OLX?',
  'Co zrobić, jeśli nie otrzymałem linku aktywacyjnego po rejestracji na OLX?',
  'Czy mogę używać jednego konta OLX do sprzedaży prywatnej i firmowej?'],
 '943a92e0': ['Jak mam zweryfikować moje konto OLX za pomocą numeru telefonu?',
  'Co zrobić, jeśli nie otrzymałem SMS z kodem weryfikacyjnym przy dodawaniu pierwszego ogłoszenia na OLX?',
  'Jak długo trzeba czekać na SMS z kodem weryfikacyjnym od OLX?',
  'Czy mogę jakoś sprawdzić, dlaczego nie dostaję kodu weryfikacyjnego SMS od OLX?',
  'Czy muszę zapłacić za SMS z kodem weryfikacyjnym od OLX?'],
 '9e41cc6b': ['Dlaczego nie mogę zweryfikować mojego nowego konta na OLX za pomocą numeru telefonu?',
  'Co oznacza komunikat informujący, że numer telefonu został już użyty na OLX?',
  'Jak mogę sprawdzić, czy mój nume

In [92]:
documents[0]

{'text': 'Krok 1. Załóż konto na OLX poprzez:\nstronę internetową OLX — kliknij Twoje konto → Załóż konto lub kliknij tutaj.\naplikację mobilną OLX — kliknij Konto → Zaloguj się → Załóż konto.\nJeśli grafika jest nieczytelna, kliknij prawy przycisk myszy i wybierz „Otwórz grafikę w nowej karcie".\nKrok 2. Wpisz swój adres e-mail.\nKrok 3. Wpisz hasło, które ma minimum 8 znaków, w tym przynajmniej jedną dużą i małą literę, jedną cyfrę oraz znak specjalny.\nKrok 4. Kliknij Zarejestruj się.\nKrok 5. Przejdź do skrzynki mailowej i w wiadomości od OLX kliknij przycisk Aktywuj konto.\nJeśli nie masz linku aktywacyjnego — napisz do nas.\nNa jednym koncie OLX nie można sprzedawać przedmiotów prywatnych i przedmiotów w ramach prowadzenia firmy. Do każdego typu sprzedaży (prywatnie, jako firma) wymagane jest oddzielne konto.\nKonto na OLX możesz założyć za pomocą konta na Facebooku. Wystarczy, że klikniesz tutaj. Następnie wybierz przycisk Kontynuuj z Facebook i postępuj zgodnie z wyświetlanymi 

In [90]:
doc_index = {d['id']: d for d in documents}

In [93]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    for q in questions:
        final_results.append((q, doc_id))

In [94]:
import pandas as pd

In [95]:
df = pd.DataFrame(final_results, columns=['question', 'document'])

In [96]:
df.to_csv('../data/ground-truth-data_2024_08_25.csv', index=False)

In [97]:
!head ../data/ground-truth-data_2024_08_25.csv

question,document
Jak mogę założyć konto na OLX przez stronę internetową?,2ddb5f16
Jakie są kroki potrzebne do założenia konta w aplikacji mobilnej OLX?,2ddb5f16
Jakie wymagania musi spełniać hasło podczas zakładania konta na OLX?,2ddb5f16
"Co zrobić, jeśli nie otrzymałem linku aktywacyjnego po rejestracji na OLX?",2ddb5f16
Czy mogę używać jednego konta OLX do sprzedaży prywatnej i firmowej?,2ddb5f16
Jak mam zweryfikować moje konto OLX za pomocą numeru telefonu?,943a92e0
"Co zrobić, jeśli nie otrzymałem SMS z kodem weryfikacyjnym przy dodawaniu pierwszego ogłoszenia na OLX?",943a92e0
Jak długo trzeba czekać na SMS z kodem weryfikacyjnym od OLX?,943a92e0
"Czy mogę jakoś sprawdzić, dlaczego nie dostaję kodu weryfikacyjnego SMS od OLX?",943a92e0


## Updating prompt

In [106]:
updated_prompt_template = """
Jesteś pracownikiem działu obsługi klienta w firmie OLX będącej platformą ogłoszeniową.
Otrzymałeś zadanie aby do kazdego ZAGADNIENIE z FAQ lub regulaminu stworzyć 5 pytań, które 
mógłby zadać klient platformy OLX.
Każde ZAGADNIENIE powinno zawierać odpowiedź na pytanie. Pytanie powinno być kompletne i nie za krótkie. Pytanie powinno być zadane tak jak mógłby to zrobić zwykły uytkownik serwisu. Jeśli to możliwe, użyj jak najmniej słów z oryginalnego ZAGADNIENIE i ODPOWIEDŹ.

Pytanie:

SEKCJA: {section}
ZAGADNIENIE: {question}
ODPOWIEDŹ: {text}

Odpowiedź powinna być w parsowalnym formacie JSON. Nie używaj bloków kodu:

["pytanie1", "pytanie2", "pytanie3", "pytanie4", "pytanie5"]
""".strip()

In [107]:
from openai import OpenAI
client = OpenAI()

In [108]:
def generate_questions_updated(doc):
    prompt = updated_prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [109]:
questions = generate_questions_updated(documents[0])

In [110]:
questions


'[\n"Jak mogę założyć konto na OLX przez stronę internetową?",\n"Jak założyć konto w aplikacji mobilnej OLX?",\n"Jakie są wymagania dotyczące hasła podczas rejestracji na OLX?",\n"Co mam zrobić, jeśli nie otrzymałem linku aktywacyjnego na mój e-mail?",\n"Czy mogę założyć konto na OLX za pomocą Facebooka i jak to zrobić?"\n]'

In [ ]:
questions


'["Jak mogę założyć konto na stronie internetowej OLX?", "Jak założyć konto na OLX za pomocą aplikacji mobilnej?", "Jakie wymagania musi spełniać hasło przy tworzeniu konta na OLX?", "Co zrobić, jeśli nie mogę znaleźć linku aktywacyjnego po rejestracji na OLX?", "Czy mogę używać jednego konta OLX do sprzedaży prywatnej i firmowej?"]'

In [111]:
from tqdm.auto import tqdm
import pickle
import pandas as pd

In [119]:
results = {}

In [120]:
documents_no_definitions = [doc for doc in documents if doc['section'] != 'Definicje']

In [121]:
for doc in tqdm(documents_no_definitions): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions_updated(doc)
    results[doc_id] = questions

  0%|          | 0/78 [00:00<?, ?it/s]

100%|██████████| 78/78 [02:31<00:00,  1.94s/it]


In [122]:
with open('../data/results_updated.bin', 'wb') as f_out:
    pickle.dump(results, f_out)

In [123]:
with open('../data/results_updated.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [124]:
results['653d264c']

'["Na jakiej podstawie OLX podejmuje decyzje o blokadzie lub zawieszeniu treści lub konta?", "Jak OLX identyfikuje treści lub działania użytkowników, które naruszają regulamin?", "Jakie są główne cele polityk i procedur moderacyjnych OLX?", "Co OLX bada podczas oceny treści pod kątem naruszenia regulaminu?", "Dlaczego OLX blokuje lub zawiesza niektóre treści lub konta użytkowników?"]'

In [125]:
import re 
parsed_results = {}

for doc_id, json_questions in results.items():
    json_questions = re.sub(r'```json', '', json_questions)
    json_questions = re.sub(r'```', '', json_questions)
    parsed_results[doc_id] = json.loads(json_questions)

In [126]:
doc_index = {d['id']: d for d in documents}

In [127]:
final_results = []

for doc_id, questions in parsed_results.items():
    for q in questions:
        final_results.append((q, doc_id))

In [128]:
df = pd.DataFrame(final_results, columns=['question', 'document'])

In [129]:
df.to_csv('../data/ground-truth-data_2024_08_30.csv', index=False)

In [130]:
!head ../data/ground-truth-data_2024_08_30.csv

question,document
Jak mogę założyć konto przez stronę internetową OLX?,2ddb5f16
"Jakie kroki muszę podjąć, żeby zarejestrować się w aplikacji mobilnej OLX?",2ddb5f16
"Co powinienem zrobić, jeśli nie widzę grafiki podczas rejestracji na OLX?",2ddb5f16
"Jakie wymagania musi spełniać hasło, gdy zakładam konto na OLX?",2ddb5f16
"Co mam zrobić, jeśli nie dostanę linku aktywacyjnego po rejestracji na OLX?",2ddb5f16
Jak mogę zweryfikować swoje konto na OLX?,943a92e0
Dlaczego nie otrzymałem SMS z kodem weryfikacyjnym?,943a92e0
Ile czasu zajmuje otrzymanie SMS z kodem weryfikacyjnym?,943a92e0
"Co mam zrobić, jeśli nie otrzymałem kodu weryfikacyjnego od OLX?",943a92e0
